In [1]:
from myutils import LRN2D
import myutils
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.engine.topology import Layer
from keras import backend as K
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
from datetime import datetime
import tensorflow as tf


face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
smile_cascade = cv2.CascadeClassifier('haarcascade_smile.xml')



Using TensorFlow backend.


In [2]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [3]:
all_images = {}

for run in os.listdir('./faces'): 
    iteration = []
    if not run.startswith('.'):
        print(run)
        for img in os.listdir('./faces/' + str(run)):
            iteration.append(img)
        all_images.update({str(run) : iteration})
all_images

2019-08-07 19:47:58.973903
2019-08-07 19:50:05.817482


{'2019-08-07 19:47:58.973903': ['4.png',
  '5.png',
  '6.png',
  '2.png',
  '3.png',
  '1.png'],
 '2019-08-07 19:50:05.817482': ['176.png',
  '88.png',
  '162.png',
  '189.png',
  '77.png',
  '63.png',
  '200.png',
  '214.png',
  '215.png',
  '201.png',
  '62.png',
  '188.png',
  '76.png',
  '163.png',
  '177.png',
  '89.png',
  '149.png',
  '161.png',
  '175.png',
  '60.png',
  '74.png',
  '48.png',
  '203.png',
  '202.png',
  '49.png',
  '75.png',
  '61.png',
  '174.png',
  '160.png',
  '148.png',
  '164.png',
  '170.png',
  '158.png',
  '59.png',
  '65.png',
  '71.png',
  '212.png',
  '206.png',
  '207.png',
  '213.png',
  '70.png',
  '64.png',
  '58.png',
  '159.png',
  '171.png',
  '165.png',
  '173.png',
  '167.png',
  '99.png',
  '8.png',
  '72.png',
  '198.png',
  '66.png',
  '205.png',
  '211.png',
  '210.png',
  '204.png',
  '199.png',
  '67.png',
  '73.png',
  '9.png',
  '166.png',
  '98.png',
  '172.png',
  '129.png',
  '115.png',
  '101.png',
  '14.png',
  '28.png',
  '29.

In [4]:
myInput = Input(shape=(96, 96, 3))

x = ZeroPadding2D(padding=(3, 3), input_shape=(96, 96, 3))(myInput)
x = Conv2D( 64, (7, 7), strides=(2, 2), name='conv1')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn1')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)
x = Lambda(LRN2D, name='lrn_1')(x)
x = Conv2D(64, (1, 1), name='conv2')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn2')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = Conv2D(192, (3, 3), name='conv3')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn3')(x)
x = Activation('relu')(x)
x = Lambda(LRN2D, name='lrn_2')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)

# Inception3a
inception_3a_3x3 = Conv2D(96, (1, 1), name='inception_3a_3x3_conv1')(x)
inception_3a_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_3x3_bn1')(inception_3a_3x3)
inception_3a_3x3 = Activation('relu')(inception_3a_3x3)
inception_3a_3x3 = ZeroPadding2D(padding=(1, 1))(inception_3a_3x3)
inception_3a_3x3 = Conv2D(128, (3, 3), name='inception_3a_3x3_conv2')(inception_3a_3x3)
inception_3a_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_3x3_bn2')(inception_3a_3x3)
inception_3a_3x3 = Activation('relu')(inception_3a_3x3)

inception_3a_5x5 = Conv2D(16, (1, 1), name='inception_3a_5x5_conv1')(x)
inception_3a_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_5x5_bn1')(inception_3a_5x5)
inception_3a_5x5 = Activation('relu')(inception_3a_5x5)
inception_3a_5x5 = ZeroPadding2D(padding=(2, 2))(inception_3a_5x5)
inception_3a_5x5 = Conv2D(32, (5, 5), name='inception_3a_5x5_conv2')(inception_3a_5x5)
inception_3a_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_5x5_bn2')(inception_3a_5x5)
inception_3a_5x5 = Activation('relu')(inception_3a_5x5)

inception_3a_pool = MaxPooling2D(pool_size=3, strides=2)(x)
inception_3a_pool = Conv2D(32, (1, 1), name='inception_3a_pool_conv')(inception_3a_pool)
inception_3a_pool = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_pool_bn')(inception_3a_pool)
inception_3a_pool = Activation('relu')(inception_3a_pool)
inception_3a_pool = ZeroPadding2D(padding=((3, 4), (3, 4)))(inception_3a_pool)

inception_3a_1x1 = Conv2D(64, (1, 1), name='inception_3a_1x1_conv')(x)
inception_3a_1x1 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_1x1_bn')(inception_3a_1x1)
inception_3a_1x1 = Activation('relu')(inception_3a_1x1)

inception_3a = concatenate([inception_3a_3x3, inception_3a_5x5, inception_3a_pool, inception_3a_1x1], axis=3)

# Inception3b
inception_3b_3x3 = Conv2D(96, (1, 1), name='inception_3b_3x3_conv1')(inception_3a)
inception_3b_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_3x3_bn1')(inception_3b_3x3)
inception_3b_3x3 = Activation('relu')(inception_3b_3x3)
inception_3b_3x3 = ZeroPadding2D(padding=(1, 1))(inception_3b_3x3)
inception_3b_3x3 = Conv2D(128, (3, 3), name='inception_3b_3x3_conv2')(inception_3b_3x3)
inception_3b_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_3x3_bn2')(inception_3b_3x3)
inception_3b_3x3 = Activation('relu')(inception_3b_3x3)

inception_3b_5x5 = Conv2D(32, (1, 1), name='inception_3b_5x5_conv1')(inception_3a)
inception_3b_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_5x5_bn1')(inception_3b_5x5)
inception_3b_5x5 = Activation('relu')(inception_3b_5x5)
inception_3b_5x5 = ZeroPadding2D(padding=(2, 2))(inception_3b_5x5)
inception_3b_5x5 = Conv2D(64, (5, 5), name='inception_3b_5x5_conv2')(inception_3b_5x5)
inception_3b_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_5x5_bn2')(inception_3b_5x5)
inception_3b_5x5 = Activation('relu')(inception_3b_5x5)

inception_3b_pool = Lambda(lambda x: x**2, name='power2_3b')(inception_3a)
inception_3b_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_3b_pool)
inception_3b_pool = Lambda(lambda x: x*9, name='mult9_3b')(inception_3b_pool)
inception_3b_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_3b')(inception_3b_pool)
inception_3b_pool = Conv2D(64, (1, 1), name='inception_3b_pool_conv')(inception_3b_pool)
inception_3b_pool = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_pool_bn')(inception_3b_pool)
inception_3b_pool = Activation('relu')(inception_3b_pool)
inception_3b_pool = ZeroPadding2D(padding=(4, 4))(inception_3b_pool)

inception_3b_1x1 = Conv2D(64, (1, 1), name='inception_3b_1x1_conv')(inception_3a)
inception_3b_1x1 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_1x1_bn')(inception_3b_1x1)
inception_3b_1x1 = Activation('relu')(inception_3b_1x1)

inception_3b = concatenate([inception_3b_3x3, inception_3b_5x5, inception_3b_pool, inception_3b_1x1], axis=3)

# Inception3c
inception_3c_3x3 = myutils.conv2d_bn(inception_3b,
                                   layer='inception_3c_3x3',
                                   cv1_out=128,
                                   cv1_filter=(1, 1),
                                   cv2_out=256,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(2, 2),
                                   padding=(1, 1))

inception_3c_5x5 = myutils.conv2d_bn(inception_3b,
                                   layer='inception_3c_5x5',
                                   cv1_out=32,
                                   cv1_filter=(1, 1),
                                   cv2_out=64,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(2, 2),
                                   padding=(2, 2))

inception_3c_pool = MaxPooling2D(pool_size=3, strides=2)(inception_3b)
inception_3c_pool = ZeroPadding2D(padding=((0, 1), (0, 1)))(inception_3c_pool)

inception_3c = concatenate([inception_3c_3x3, inception_3c_5x5, inception_3c_pool], axis=3)

#inception 4a
inception_4a_3x3 = myutils.conv2d_bn(inception_3c,
                                   layer='inception_4a_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=192,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
inception_4a_5x5 = myutils.conv2d_bn(inception_3c,
                                   layer='inception_4a_5x5',
                                   cv1_out=32,
                                   cv1_filter=(1, 1),
                                   cv2_out=64,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(1, 1),
                                   padding=(2, 2))

inception_4a_pool = Lambda(lambda x: x**2, name='power2_4a')(inception_3c)
inception_4a_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_4a_pool)
inception_4a_pool = Lambda(lambda x: x*9, name='mult9_4a')(inception_4a_pool)
inception_4a_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_4a')(inception_4a_pool)
inception_4a_pool = myutils.conv2d_bn(inception_4a_pool,
                                   layer='inception_4a_pool',
                                   cv1_out=128,
                                   cv1_filter=(1, 1),
                                   padding=(2, 2))
inception_4a_1x1 = myutils.conv2d_bn(inception_3c,
                                   layer='inception_4a_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))
inception_4a = concatenate([inception_4a_3x3, inception_4a_5x5, inception_4a_pool, inception_4a_1x1], axis=3)

#inception4e
inception_4e_3x3 = myutils.conv2d_bn(inception_4a,
                                   layer='inception_4e_3x3',
                                   cv1_out=160,
                                   cv1_filter=(1, 1),
                                   cv2_out=256,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(2, 2),
                                   padding=(1, 1))
inception_4e_5x5 = myutils.conv2d_bn(inception_4a,
                                   layer='inception_4e_5x5',
                                   cv1_out=64,
                                   cv1_filter=(1, 1),
                                   cv2_out=128,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(2, 2),
                                   padding=(2, 2))
inception_4e_pool = MaxPooling2D(pool_size=3, strides=2)(inception_4a)
inception_4e_pool = ZeroPadding2D(padding=((0, 1), (0, 1)))(inception_4e_pool)

inception_4e = concatenate([inception_4e_3x3, inception_4e_5x5, inception_4e_pool], axis=3)

#inception5a
inception_5a_3x3 = myutils.conv2d_bn(inception_4e,
                                   layer='inception_5a_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=384,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))

inception_5a_pool = Lambda(lambda x: x**2, name='power2_5a')(inception_4e)
inception_5a_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_5a_pool)
inception_5a_pool = Lambda(lambda x: x*9, name='mult9_5a')(inception_5a_pool)
inception_5a_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_5a')(inception_5a_pool)
inception_5a_pool = myutils.conv2d_bn(inception_5a_pool,
                                   layer='inception_5a_pool',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   padding=(1, 1))
inception_5a_1x1 = myutils.conv2d_bn(inception_4e,
                                   layer='inception_5a_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))

inception_5a = concatenate([inception_5a_3x3, inception_5a_pool, inception_5a_1x1], axis=3)

#inception_5b
inception_5b_3x3 = myutils.conv2d_bn(inception_5a,
                                   layer='inception_5b_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=384,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
inception_5b_pool = MaxPooling2D(pool_size=3, strides=2)(inception_5a)
inception_5b_pool = myutils.conv2d_bn(inception_5b_pool,
                                   layer='inception_5b_pool',
                                   cv1_out=96,
                                   cv1_filter=(1, 1))
inception_5b_pool = ZeroPadding2D(padding=(1, 1))(inception_5b_pool)

inception_5b_1x1 = myutils.conv2d_bn(inception_5a,
                                   layer='inception_5b_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))
inception_5b = concatenate([inception_5b_3x3, inception_5b_pool, inception_5b_1x1], axis=3)

av_pool = AveragePooling2D(pool_size=(3, 3), strides=(1, 1))(inception_5b)
reshape_layer = Flatten()(av_pool)
dense_layer = Dense(128, name='dense_layer')(reshape_layer)
norm_layer = Lambda(lambda  x: K.l2_normalize(x, axis=1), name='norm_layer')(dense_layer)


# Final Model
model = Model(inputs=[myInput], outputs=norm_layer)

model.summary()


W0812 11:33:04.532402 4638381504 deprecation_wrapper.py:119] From /Users/joshuakang/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0812 11:33:04.553736 4638381504 deprecation_wrapper.py:119] From /Users/joshuakang/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0812 11:33:04.567522 4638381504 deprecation_wrapper.py:119] From /Users/joshuakang/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0812 11:33:04.603440 4638381504 deprecation_wrapper.py:119] From /Users/joshuakang/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_se

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 102, 102, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 48, 48, 64)   9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn1 (BatchNormalization)        (None, 48, 48, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [5]:
# Load weights from csv files (which was exported from Openface torch model)
weights = myutils.weights
weights_dict = myutils.load_weights()

# Set layer weights of the model
for name in weights:
  if model.get_layer(name) != None:
    model.get_layer(name).set_weights(weights_dict[name])
  elif model.get_layer(name) != None:
    model.get_layer(name).set_weights(weights_dict[name])

In [6]:
def image_to_embedding(image, model):
    #image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_AREA) 
    image = cv2.resize(image, (96, 96)) 
    img = image[...,::-1]
    img = np.around(np.transpose(img, (0,1,2))/255.0, decimals=12)
    x_train = np.array([img])
    embedding = model.predict_on_batch(x_train)
#     print ('embedding from model : ' + str(embedding))
    return embedding

In [1]:
def recognize_face(face_image, input_embeddings, model):

    captured_embedding = image_to_embedding(face_image, model)
    
    minimum_distance = 200
    name = None
    
    # Loop over  names and encodings.
#     for (input_name, input_embedding) in input_embeddings.items():
    for input_name in input_embeddings:
        sum = 0
        for input_embedding in input_embeddings[input_name]: 

            
            euclidean_distance = np.linalg.norm(captured_embedding-input_embedding)

#             print('Euclidean distance from %s is %s' %(input_name, euclidean_distance))
            sum += euclidean_distance
        
            if euclidean_distance < minimum_distance:
                minimum_distance = euclidean_distance
                name = input_name
                
        average = float(euclidean_distance)/float(len(input_embeddings[input_name]))
        print('average distance to Josh: ' + str(average))
    
    if minimum_distance < 0.68:
        return str(name)
    else:
        return "Not Josh"

SyntaxError: invalid syntax (<ipython-input-1-d88fa5c40972>, line 24)

In [2]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

embeddings = {}
embeddings['Josh'] = []

for run in all_images: 
    images = all_images[run]
    for image in images: 
        image_file = cv2.imread('./faces/' + run + '/' + image)
        embeddings['Josh'].append(image_to_embedding(image_file, model))

embeddings['Josh']


Using TensorFlow backend.


NameError: name 'all_images' is not defined

In [9]:
#gray is the input frame converted into gray scale for the facial recognition 
#frame is returned with rectangles drawn if faces were found
def detect(gray, frame, faceCount, rightNow, embeddings, model):

    #we require that there must be 5 neighboring accepted features to be classified true
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces : 
        
        #region of interest to detect the eyes 
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        
        #1.1 is the zoom from the original given image "roi_gray"
        eyes = eye_cascade.detectMultiScale(roi_gray, 1.1, 3)
        
        #finding the eyes within the already classified face images to save compute power
        face_eyes = []
        for (ex, ey, ew, eh) in eyes : 
            single_eye = {'ex': ex, 'ey': ey, 'ew': ew, 'eh': eh}
            
            face_eyes.append(single_eye)
            
        #this mean that we have found a valid face
        if len(face_eyes) == 2: 
            faceCount += 1

            print("2 eyes appearred means a valid face!")
            identity = recognize_face(frame[y:y+h, x:x+w], embeddings, model)

        for eye in face_eyes: 
            #top left coordinates, bottom right coordinates, green, 2 thickness
            print("current eye : " + str(eye))
            cv2.rectangle(roi_color, (eye['ex'], eye['ey']), (eye['ex'] + eye['ew'], eye['ey'] + eye['eh']), (0, 255, 0), 2)

       
        #top left coordinates, bottom right coordinates, red, 2 thickness
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        
        smiles = smile_cascade.detectMultiScale(roi_gray, 1.7, 22)
        for (sx, sy, sw, sh) in smiles:
            cv2.rectangle(roi_color, (sx, sy), (sx+sw, sy+sh), (0, 0, 255), 2)
    
    return frame, faceCount
        
    

In [10]:
video_capture = cv2.VideoCapture(0)
faceCount = 0
rightNow = datetime.now()


In [ ]:
while True : 
    
    _, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    canvas, faceCount = detect(gray, frame, faceCount, rightNow, embeddings, model)
    cv2.imshow('Video', canvas)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        

2 eyes appearred means a valid face!
Euclidean distance from Josh is 0.63569254
Euclidean distance from Josh is 0.6498764
Euclidean distance from Josh is 0.5554394
Euclidean distance from Josh is 0.65032125
Euclidean distance from Josh is 0.6356939
Euclidean distance from Josh is 0.6449553
Euclidean distance from Josh is 0.8132538
Euclidean distance from Josh is 0.6205812
Euclidean distance from Josh is 0.78418386
Euclidean distance from Josh is 0.6548674
Euclidean distance from Josh is 0.7517348
Euclidean distance from Josh is 0.72061604
Euclidean distance from Josh is 0.7509052
Euclidean distance from Josh is 0.61337996
Euclidean distance from Josh is 0.61268884
Euclidean distance from Josh is 0.7216014
Euclidean distance from Josh is 0.7500982
Euclidean distance from Josh is 0.7748603
Euclidean distance from Josh is 0.6995928
Euclidean distance from Josh is 0.76339304
Euclidean distance from Josh is 0.83677495
Euclidean distance from Josh is 0.4767748
Euclidean distance from Josh is

2 eyes appearred means a valid face!
Euclidean distance from Josh is 0.8152403
Euclidean distance from Josh is 0.84140456
Euclidean distance from Josh is 0.79119146
Euclidean distance from Josh is 0.83310235
Euclidean distance from Josh is 0.832388
Euclidean distance from Josh is 0.82880837
Euclidean distance from Josh is 0.951317
Euclidean distance from Josh is 0.8453773
Euclidean distance from Josh is 0.94586
Euclidean distance from Josh is 0.8527042
Euclidean distance from Josh is 0.8274551
Euclidean distance from Josh is 0.83534867
Euclidean distance from Josh is 0.9437847
Euclidean distance from Josh is 0.8066089
Euclidean distance from Josh is 0.84989893
Euclidean distance from Josh is 0.9257233
Euclidean distance from Josh is 0.84858626
Euclidean distance from Josh is 0.89506996
Euclidean distance from Josh is 0.8618605
Euclidean distance from Josh is 0.91954285
Euclidean distance from Josh is 0.96806824
Euclidean distance from Josh is 0.7778307
Euclidean distance from Josh is 0

2 eyes appearred means a valid face!
Euclidean distance from Josh is 0.6983242
Euclidean distance from Josh is 0.71196926
Euclidean distance from Josh is 0.7518744
Euclidean distance from Josh is 0.7810949
Euclidean distance from Josh is 0.77609116
Euclidean distance from Josh is 0.76711094
Euclidean distance from Josh is 0.7129275
Euclidean distance from Josh is 0.81867796
Euclidean distance from Josh is 0.75457776
Euclidean distance from Josh is 0.73337114
Euclidean distance from Josh is 0.59484154
Euclidean distance from Josh is 0.748992
Euclidean distance from Josh is 0.846788
Euclidean distance from Josh is 0.7154742
Euclidean distance from Josh is 0.770248
Euclidean distance from Josh is 0.8231527
Euclidean distance from Josh is 0.7575285
Euclidean distance from Josh is 0.7069953
Euclidean distance from Josh is 0.6791678
Euclidean distance from Josh is 0.7385779
Euclidean distance from Josh is 0.75158775
Euclidean distance from Josh is 0.7158743
Euclidean distance from Josh is 0.

current eye : {'ex': 66, 'ey': 34, 'ew': 24, 'eh': 24}
current eye : {'ex': 23, 'ey': 30, 'ew': 43, 'eh': 43}
current eye : {'ex': 81, 'ey': 40, 'ew': 38, 'eh': 38}
2 eyes appearred means a valid face!
Euclidean distance from Josh is 0.5855838
Euclidean distance from Josh is 0.62055427
Euclidean distance from Josh is 0.536606
Euclidean distance from Josh is 0.40332973
Euclidean distance from Josh is 0.6041261
Euclidean distance from Josh is 0.53593034
Euclidean distance from Josh is 0.78901833
Euclidean distance from Josh is 0.37353373
Euclidean distance from Josh is 0.6291072
Euclidean distance from Josh is 0.5120813
Euclidean distance from Josh is 0.69907427
Euclidean distance from Josh is 0.66026926
Euclidean distance from Josh is 0.4440935
Euclidean distance from Josh is 0.45088345
Euclidean distance from Josh is 0.4512531
Euclidean distance from Josh is 0.42109737
Euclidean distance from Josh is 0.6834482
Euclidean distance from Josh is 0.69697183
Euclidean distance from Josh is 0

2 eyes appearred means a valid face!
Euclidean distance from Josh is 0.59163773
Euclidean distance from Josh is 0.6109627
Euclidean distance from Josh is 0.5813429
Euclidean distance from Josh is 0.43849272
Euclidean distance from Josh is 0.64253604
Euclidean distance from Josh is 0.5466781
Euclidean distance from Josh is 0.70399696
Euclidean distance from Josh is 0.40783745
Euclidean distance from Josh is 0.5410508
Euclidean distance from Josh is 0.43384406
Euclidean distance from Josh is 0.6701667
Euclidean distance from Josh is 0.65599805
Euclidean distance from Josh is 0.42275578
Euclidean distance from Josh is 0.42335197
Euclidean distance from Josh is 0.43420154
Euclidean distance from Josh is 0.40349898
Euclidean distance from Josh is 0.6792891
Euclidean distance from Josh is 0.65112495
Euclidean distance from Josh is 0.6101831
Euclidean distance from Josh is 0.5624435
Euclidean distance from Josh is 0.7664509
Euclidean distance from Josh is 0.56257534
Euclidean distance from Jo

...

current eye : {'ex': 168, 'ey': 216, 'ew': 22, 'eh': 22}
current eye : {'ex': 87, 'ey': 218, 'ew': 26, 'eh': 26}
current eye : {'ex': 155, 'ey': 69, 'ew': 72, 'eh': 72}
current eye : {'ex': 45, 'ey': 74, 'ew': 73, 'eh': 73}
current eye : {'ex': 54, 'ey': 78, 'ew': 74, 'eh': 74}
current eye : {'ex': 166, 'ey': 75, 'ew': 69, 'eh': 69}
current eye : {'ex': 103, 'ey': 196, 'ew': 91, 'eh': 91}
2 eyes appearred means a valid face!
Euclidean distance from Josh is 0.595249
Euclidean distance from Josh is 0.60200495
Euclidean distance from Josh is 0.51285374
Euclidean distance from Josh is 0.48190773
Euclidean distance from Josh is 0.60201573
Euclidean distance from Josh is 0.51913613
Euclidean distance from Josh is 0.7974905
Euclidean distance from Josh is 0.36693093
Euclidean distance from Josh is 0.6398426
Euclidean distance from Josh is 0.44113487
Euclidean distance from Josh is 0.63836247
Euclidean distance from Josh is 0.60092366
Euclidean distance from Josh is 0.5019092
Euclidean distanc

2 eyes appearred means a valid face!
Euclidean distance from Josh is 0.6600792
Euclidean distance from Josh is 0.6673752
Euclidean distance from Josh is 0.54904324
Euclidean distance from Josh is 0.5430386
Euclidean distance from Josh is 0.64126474
Euclidean distance from Josh is 0.5807801
Euclidean distance from Josh is 0.8663107
Euclidean distance from Josh is 0.454516
Euclidean distance from Josh is 0.7223974
Euclidean distance from Josh is 0.5455173
Euclidean distance from Josh is 0.6729838
Euclidean distance from Josh is 0.5962834
Euclidean distance from Josh is 0.61671585
Euclidean distance from Josh is 0.5144083
Euclidean distance from Josh is 0.46547058
Euclidean distance from Josh is 0.5734387
Euclidean distance from Josh is 0.5838454
Euclidean distance from Josh is 0.67950636
Euclidean distance from Josh is 0.6483654
Euclidean distance from Josh is 0.7250513
Euclidean distance from Josh is 0.93272036
Euclidean distance from Josh is 0.60663176
Euclidean distance from Josh is 0

current eye : {'ex': 103, 'ey': 168, 'ew': 45, 'eh': 45}
current eye : {'ex': 55, 'ey': 83, 'ew': 71, 'eh': 71}
current eye : {'ex': 156, 'ey': 82, 'ew': 75, 'eh': 75}
current eye : {'ex': 147, 'ey': 175, 'ew': 44, 'eh': 44}
current eye : {'ex': 108, 'ey': 175, 'ew': 43, 'eh': 43}
current eye : {'ex': 59, 'ey': 92, 'ew': 71, 'eh': 71}
current eye : {'ex': 166, 'ey': 90, 'ew': 72, 'eh': 72}
current eye : {'ex': 173, 'ey': 95, 'ew': 64, 'eh': 64}
current eye : {'ex': 53, 'ey': 90, 'ew': 79, 'eh': 79}
current eye : {'ex': 100, 'ey': 180, 'ew': 56, 'eh': 56}
current eye : {'ex': 58, 'ey': 94, 'ew': 72, 'eh': 72}
current eye : {'ex': 111, 'ey': 189, 'ew': 45, 'eh': 45}
current eye : {'ex': 171, 'ey': 90, 'ew': 80, 'eh': 80}
current eye : {'ex': 64, 'ey': 103, 'ew': 70, 'eh': 70}
current eye : {'ex': 186, 'ey': 97, 'ew': 71, 'eh': 71}
current eye : {'ex': 34, 'ey': 49, 'ew': 135, 'eh': 135}
2 eyes appearred means a valid face!
Euclidean distance from Josh is 0.64250207
Euclidean distance fro

current eye : {'ex': 128, 'ey': 213, 'ew': 45, 'eh': 45}
current eye : {'ex': 200, 'ey': 100, 'ew': 69, 'eh': 69}
current eye : {'ex': 67, 'ey': 103, 'ew': 75, 'eh': 75}
2 eyes appearred means a valid face!
Euclidean distance from Josh is 0.7086921
Euclidean distance from Josh is 0.7328265
Euclidean distance from Josh is 0.6311662
Euclidean distance from Josh is 0.6485664
Euclidean distance from Josh is 0.73998255
Euclidean distance from Josh is 0.68362606
Euclidean distance from Josh is 0.88042647
Euclidean distance from Josh is 0.49789765
Euclidean distance from Josh is 0.7681239
Euclidean distance from Josh is 0.5678523
Euclidean distance from Josh is 0.7114374
Euclidean distance from Josh is 0.76984364
Euclidean distance from Josh is 0.5147715
Euclidean distance from Josh is 0.56064314
Euclidean distance from Josh is 0.5738456
Euclidean distance from Josh is 0.51832795
Euclidean distance from Josh is 0.74607956
Euclidean distance from Josh is 0.8017228
Euclidean distance from Josh 

2 eyes appearred means a valid face!
Euclidean distance from Josh is 0.8753839
Euclidean distance from Josh is 0.9026316
Euclidean distance from Josh is 0.85556686
Euclidean distance from Josh is 0.83448416
Euclidean distance from Josh is 0.96237916
Euclidean distance from Josh is 0.8827728
Euclidean distance from Josh is 0.85519695
Euclidean distance from Josh is 0.7744136
Euclidean distance from Josh is 0.8284736
Euclidean distance from Josh is 0.739331
Euclidean distance from Josh is 0.8814641
Euclidean distance from Josh is 0.9788048
Euclidean distance from Josh is 0.62060386
Euclidean distance from Josh is 0.77178633
Euclidean distance from Josh is 0.85075825
Euclidean distance from Josh is 0.66522497
Euclidean distance from Josh is 0.9763621
Euclidean distance from Josh is 0.9328628
Euclidean distance from Josh is 0.88252527
Euclidean distance from Josh is 0.85023254
Euclidean distance from Josh is 0.80874836
Euclidean distance from Josh is 0.72786087
Euclidean distance from Josh